# ROL - WaveBot


In [1]:
import autograd.numpy as np
import capytaine as cpy
import matplotlib.pyplot as plt

import wecopttool as wot

plt.style.use('tableau-colorblind10')

### Waves

In [2]:
wavefreq = 0.3 # Hz
f1 = wavefreq
nfreq = 10
freq = wot.frequency(f1, nfreq, False)

amplitude = 0.0625 # m
phase = 30 # degrees
wavedir = 0 # degrees

waves = wot.waves.regular_wave(f1, nfreq, wavefreq, amplitude, phase, wavedir)

### WEC object

In [3]:
wb = wot.geom.WaveBot()
mesh_size_factor = 0.5
mesh = wb.mesh(mesh_size_factor)
fb = cpy.FloatingBody.from_meshio(mesh, name="WaveBot")

fb.add_translation_dof(name="Heave")
ndof = fb.nb_dofs

bem_data = wot.run_bem(fb, freq)

name = ["PTO_Heave",]
kinematics = np.eye(ndof)
controller = None
loss = None
pto_impedance = None
pto = wot.pto.PTO(ndof, kinematics, controller, pto_impedance, loss, name)

f_add = {'PTO': pto.force_on_wec}

wec = wot.WEC.from_bem(bem_data, f_add=f_add)

[14:56:40] WARNING  Using the geometric centroid as the center of gravity (COG).

           WARNING  Using the center of gravity (COG) as the rotation center for hydrostatics.

The resolution of the mesh might be insufficient for omega ranging from 9.425 to 18.850.
This warning appears when the largest panel of this mesh has radius > wavelength/8.


           WARNING  Mesh resolution for 12 problems:                                                               
                    The resolution of the mesh might be insufficient for omega ranging from 9.425 to 18.850.       
                    This warning appears when the largest panel of this mesh has radius > wavelength/8.

Output()

[14:56:43] WARNING  Linear damping for DOF "Heave" has negative or close to zero terms. Shifting up damping terms  
                    [5 7 8 9] to a minimum of 1e-06 N/(m/s)

### Objective function

In [4]:
obj_fun = pto.mechanical_average_power
nstate_opt = 2*nfreq

### Solve

In [5]:
scale_x_wec = 1 # 1e1
scale_x_opt = 1 # 1e-3
scale_obj = 1 # 1e-2

results = wec.solve(
    waves,
    obj_fun,
    nstate_opt,
    scale_x_wec=scale_x_wec,
    scale_x_opt=scale_x_opt,
    scale_obj=scale_obj,
    parameter_file='parameters.xml',
    )

opt_mechanical_average_power = results[0]['f']
print(f'Optimal average mechanical power: {opt_mechanical_average_power} W')

/Users/cmichel/repos/WecOptTool/wecopttool/core.py:712: UserWarning: The `squeeze` kwarg to GroupBy is being removed.Pass .groupby(..., squeeze=False) to disable squeezing, which is the new default, and to silence this warning.
  for realization, wave in waves.groupby('realization'):



  Check primal optimization space vector

********** Begin verification of linear algebra. *********************************************

Commutativity of addition. Consistency error: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> 0.000000000000e+00
Associativity of addition. Consistency error: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> 4.477603233994e-16
Identity element of addition. Consistency error: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> 0.000000000000e+00
Inverse elements of addition. Consistency error: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> 0.000000000000e+00
Identity element of scalar multiplication. Consistency error: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> 0.000000000000e+00
Consistency of scalar multiplication with field multiplication. Consistency error: >>>>>>>>>>> 3.622873524014e-16
Distributivity of scalar multiplication with respect to field addition. Consistency error: >>> 4.994226407822e-16
Distributivity of scalar multiplication with respect to vector 

/Users/cmichel/miniforge3/envs/rol/lib/python3.11/site-packages/pyrol/vectors/NumPyVector.py:20: RuntimeWarning: invalid value encountered in add
  self.array += b.array
/Users/cmichel/miniforge3/envs/rol/lib/python3.11/site-packages/pyrol/vectors/NumPyVector.py:20: RuntimeWarning: invalid value encountered in add
  self.array += b.array
/Users/cmichel/miniforge3/envs/rol/lib/python3.11/site-packages/pyrol/vectors/NumPyVector.py:20: RuntimeWarning: invalid value encountered in add
  self.array += b.array
/Users/cmichel/miniforge3/envs/rol/lib/python3.11/site-packages/pyrol/vectors/NumPyVector.py:20: RuntimeWarning: invalid value encountered in add
  self.array += b.array
/Users/cmichel/miniforge3/envs/rol/lib/python3.11/site-packages/pyrol/vectors/NumPyVector.py:20: RuntimeWarning: invalid value encountered in add
  self.array += b.array
/Users/cmichel/miniforge3/envs/rol/lib/python3.11/site-packages/pyrol/vectors/NumPyVector.py:20: RuntimeWarning: invalid value encountered in add
  se

  1     -1.557275e+175 4.695968e+92   2.684250e+92   2.825878e+87   5.21e+00  1.26e-01  1.00e-01  1003    690     1003    1000    


/Users/cmichel/miniforge3/envs/rol/lib/python3.11/site-packages/pyrol/vectors/NumPyVector.py:20: RuntimeWarning: invalid value encountered in add
  self.array += b.array
/Users/cmichel/miniforge3/envs/rol/lib/python3.11/site-packages/pyrol/vectors/NumPyVector.py:20: RuntimeWarning: invalid value encountered in add
  self.array += b.array
/Users/cmichel/miniforge3/envs/rol/lib/python3.11/site-packages/pyrol/vectors/NumPyVector.py:20: RuntimeWarning: invalid value encountered in add
  self.array += b.array
/Users/cmichel/miniforge3/envs/rol/lib/python3.11/site-packages/pyrol/vectors/NumPyVector.py:20: RuntimeWarning: invalid value encountered in add
  self.array += b.array
/Users/cmichel/miniforge3/envs/rol/lib/python3.11/site-packages/pyrol/vectors/NumPyVector.py:20: RuntimeWarning: invalid value encountered in add
  self.array += b.array
/Users/cmichel/miniforge3/envs/rol/lib/python3.11/site-packages/pyrol/vectors/NumPyVector.py:20: RuntimeWarning: invalid value encountered in add
  se

  2     -1.605206e+304 inf            3.737724e+156  1.205779e+152  5.21e+01  1.26e-01  1.00e-01  2005    1442    2005    1000    
Optimal average mechanical power: -1.605205886204358e+304 W  3     -1.605206e+304 inf            8.744654e+157  0.000000e+00   5.21e+02  2.85e-02  1.92e-02  2273    1443    2273    266     
Optimization Terminated with Status: Step Tolerance Met



### Analyzing results

In [9]:
results

[{'x': array([-2.97832799e+138,  8.24597419e+142,  7.85120156e+142,
         -1.79943983e+147, -9.03845761e+146, -3.23940590e+150,
         -3.95850572e+150, -3.69596899e+151, -1.07787273e+150,
          2.78776577e+150,  7.53590208e+149,  3.02253575e+148,
          1.39157777e+148, -1.26005107e+145, -3.31382075e+145,
         -1.12974783e+144, -1.49900924e+144,  4.82929067e+142,
          1.88775224e+142, -4.19218626e+146, -3.00232034e+136,
          1.29693361e+143, -1.36197772e+143, -9.25929105e+146,
          1.84339555e+147, -4.82971692e+150,  3.95249924e+150,
         -3.27227505e+150,  1.12360608e+152,  5.61101449e+150,
         -2.07617974e+151,  2.05825590e+149, -4.47148111e+149,
         -8.46690619e+146,  3.22037599e+146, -6.01035726e+145,
          4.53082124e+145,  1.11464956e+144, -2.85258228e+144,
         -7.15575894e+140]),
  'f': -1.605205886204358e+304}]